# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [4]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [5]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [7]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [12]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [13]:
document = ET.parse( './data/mondial_database.xml' )

In [33]:
for country in document.getroot().iterfind('country'):
    if country.find('infant_mortality') == None:
        continue
    print(country.find('name').text + " " + country.find('infant_mortality').text)

Albania 13.19
Greece 4.78
Macedonia 7.9
Serbia 6.16
Andorra 3.69
France 3.31
Spain 3.33
Austria 4.16
Czech Republic 2.63
Germany 3.46
Hungary 5.09
Italy 3.31
Liechtenstein 4.33
Slovakia 5.35
Slovenia 4.04
Switzerland 3.73
Belarus 3.64
Latvia 7.91
Lithuania 6
Poland 6.19
Ukraine 8.1
Russia 7.08
Belgium 4.18
Luxembourg 4.28
Netherlands 3.66
Bosnia and Herzegovina 5.84
Croatia 5.87
Bulgaria 15.08
Romania 10.16
Turkey 21.43
Denmark 4.1
Estonia 6.7
Faroe Islands 5.71
Finland 3.36
Norway 2.48
Sweden 2.6
Monaco 1.81
Gibraltar 6.29
Guernsey 3.47
Iceland 3.15
Ireland 3.74
San Marino 4.52
Jersey 3.86
Malta 3.59
Isle of Man 4.17
Moldova 12.93
Portugal 4.48
United Kingdom 4.44
Afghanistan 117.23
China 14.79
Iran 39
Pakistan 57.48
Tajikistan 35.03
Turkmenistan 38.13
Uzbekistan 19.84
Armenia 13.97
Georgia 16.68
Azerbaijan 26.67
Bahrain 9.68
Bangladesh 45.67
Myanmar 44.91
India 43.19
Bhutan 37.89
Brunei 10.48
Malaysia 13.69
Laos 54.53
Thailand 9.86
Cambodia 51.36
Vietnam 18.99
Kazakhstan 21.61
North 

In [34]:
import pandas as pd

In [43]:
infant_mortality = pd.DataFrame(columns=['Country', 'Infant Mortality'])
infant_mortality

,Country,Infant Mortality


In [63]:

for country in document.getroot().iterfind('country'):
    if country.find('infant_mortality') == None:
        continue
    temp = pd.DataFrame([[country.find('name').text,country.find('infant_mortality').text]],columns=['Country','Infant Mortality'])
    infant_mortality = infant_mortality.append(temp)

In [66]:
infant_mortality['Infant Mortality'] = infant_mortality['Infant Mortality'].astype(float)

In [71]:
infant_mortality.sort_values('Infant Mortality', ascending=False).head(10)

,Country,Infant Mortality
0,Western Sahara,145.82
0,Afghanistan,117.23
0,Mali,104.34
0,Somalia,100.14
0,Central African Republic,92.86
0,Guinea-Bissau,90.92
0,Chad,90.30
0,Niger,86.27
0,Angola,79.99
0,Burkina Faso,76.80


In [86]:
city_population = pd.DataFrame(columns=['City', 'Population'])
city_population

,City,Population


In [87]:
for country in document.getroot().iterfind('country'):
    for city in country.iterfind('city'):
        if city.find('population') == None:
            continue
        temp = pd.DataFrame([[city.find('name').text,city.findall('population')[len(city.findall('population'))-1].text]],columns=['City','Population'])
        city_population = city_population.append(temp)
        

In [91]:
city_population['Population'] = city_population['Population'].astype(int)
city_population.sort_values('Population', ascending=False).head(10)

,City,Population
0,Seoul,9708483
0,Al Qahirah,8471859
0,Bangkok,7506700
0,Hong Kong,7055071
0,Ho Chi Minh,5968384
0,Singapore,5076700
0,Al Iskandariyah,4123869
0,New Taipei,3939305
0,Busan,3403135
0,Pyongyang,3255288


In [104]:
(int(document.getroot().find('country').find('ethnicgroup').attrib['percentage'])/100)

0.95

In [108]:
int(document.getroot().find('country').findall('population')[len(document.getroot().find('country').findall('population'))-1].text)

2800138

In [113]:
ethnic_groups = pd.DataFrame(columns=['Ethnicity','Percentage','Country Population'])
ethnic_groups

,Ethnicity,Percentage,Country Population


In [119]:
for country in document.getroot().iterfind('country'):
    for ethnicity in country.iterfind('ethnicgroup'):
        if country.find('population') == None:
            continue
        temp = pd.DataFrame([[ethnicity.text,float(ethnicity.attrib['percentage'])/100,country.findall('population')[len(country.findall('population'))-1].text]],
                           columns=['Ethnicity','Percentage','Country Population'])
        ethnic_groups = ethnic_groups.append(temp)


In [123]:
ethnic_groups['Percentage'] = ethnic_groups['Percentage'].astype(float)
ethnic_groups['Country Population'] = ethnic_groups['Country Population'].astype(int)
ethnic_groups['Ethnic Population'] = ethnic_groups['Percentage']*ethnic_groups['Country Population']

In [135]:
ethnic_groups['Ethnic Population'] = ethnic_groups['Ethnic Population'].astype(int)
ethnic_groups

,Ethnicity,Percentage,Country Population,Ethnic Population
0,Albanian,0.9500,2800138,2660131
0,Greek,0.0300,2800138,84004
0,Greek,0.9300,7632801,7098504
0,Macedonian,0.6420,1935034,1242291
0,Albanian,0.2520,1935034,487628
0,Turkish,0.0390,1935034,75466
0,Gypsy,0.0270,1935034,52245
0,Serb,0.0180,1935034,34830
0,Albanian,0.9500,2800138,2660131
0,Greek,0.0300,2800138,84004


In [137]:
ethnic_groups.groupby(['Ethnicity',]).sum().sort_values('Ethnic Population',ascending=False)

,Percentage,Country Population,Ethnic Population
Ethnicity,,,
Han Chinese,0.9150,1360720000,1245058800
Indo-Aryan,0.7200,1210854977,871815583
European,9.7082,1157295639,494872201
African,18.6855,975352746,318325104
Dravidian,0.2500,1210854977,302713744
Mestizo,8.7070,279743964,157734349
Bengali,0.9800,149772364,146776916
Russian,2.2410,322438406,131856989
Japanese,0.9940,127298000,126534212


In [138]:
ethnic_groups['Ethnic Population'].sum()

5983854907

In [160]:
print(document.getroot().find('river').find('name').text)
country_abbrev = document.getroot().find('river').find('source').attrib['country']
print(int(document.getroot().find('river').find('length').text))

for country in document.getroot().iterfind('country'):
    if country.attrib['car_code'] == country_abbrev:
        print(country.find('name').text)

Thjorsa
230
Iceland


In [162]:
rivers = pd.DataFrame(columns=['Country','River Name','Length'])
rivers

,Country,River Name,Length


In [166]:
for river in document.getroot().iterfind('river'):
    if river.find('length')==None:
        continue
    country_abbrev = river.find('source').attrib['country']
    river_name = river.find('name').text
    length = float(river.find('length').text)
    for country in document.getroot().iterfind('country'):
        if country.attrib['car_code'] == country_abbrev:
            country_name = country.find('name').text
    temp = pd.DataFrame([[country_name,river_name,length]],columns=['Country','River Name','Length'])
    rivers = rivers.append(temp)

In [170]:
rivers.sort_values('Length',ascending=False).head(1)

,Country,River Name,Length
0,Peru,Amazonas,6448.0


In [177]:
print(document.getroot().find('lake').find('name').text)
print(document.getroot().find('lake').find('area').text)
country_abbrev = document.getroot().find('lake').attrib['country']
for country in document.getroot().iterfind('country'):
    if country.attrib['car_code'] == country_abbrev:
        print(country.find('name').text)

Inari
1040
Finland


In [190]:
lakes_df1 = pd.DataFrame(columns=['Country','Lake','Area'])
lakes_df1

,Country,Lake,Area


In [191]:
for lake in document.getroot().iterfind('lake'):
    if lake.find('area') == None:
        continue
    lake_name = lake.find('name').text
    lake_area = float(lake.find('area').text)
    country_abbrev = lake.attrib['country']
    for country in document.getroot().iterfind('country'):
        if country.attrib['car_code'] == country_abbrev:
            country_name = country.find('name').text
    temp = pd.DataFrame([[country_name,lake_name,lake_area]],columns=['Country','Lake','Area'])
    lakes_df1 = lakes_df1.append(temp)

In [194]:
lakes_df1.sort_values('Area',ascending=False).head(1)

,Country,Lake,Area
0,Iran,Caspian Sea,386400.0


In [198]:
print(document.getroot().find('airport').find('name').text)
print(document.getroot().find('airport').find('elevation').text)
print(document.getroot().find('airport').attrib['country'])

Herat
977
AFG


In [199]:
airports = pd.DataFrame(columns=['Country','Airport','Elevation'])
airports

,Country,Airport,Elevation


In [204]:
for airport in document.getroot().iterfind('airport'):
    if airport.find('elevation')==None:
        continue
    airport_name = airport.find('name').text
    elevation = airport.find('elevation').text
    country_abbrev = airport.attrib['country']
    for country in document.getroot().iterfind('country'):
        if country.attrib['car_code'] == country_abbrev:
            country_name = country.find('name').text
    temp = pd.DataFrame([[country_name,airport_name,elevation]],columns=['Country','Airport','Elevation'])
    airports = airports.append(temp)
            

In [206]:
airports['Elevation'] = airports['Elevation'].astype(float)

In [209]:
airports.sort_values('Elevation',ascending=False).head(1)

,Country,Airport,Elevation
0,Bolivia,El Alto Intl,4063.0
